In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch

np.random.seed(17)


def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    i=-1
    for ann in sorted_anns:
        i = i+1
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [1]])
        img[m] = color_mask
    ax.imshow(img)


In [ ]:
# upload 0-original in OBJ-GSP
image = cv2.imread('images/0-original.png')

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

sys.path.append("..")
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = torch.device("cuda")
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam,min_mask_region_area=10000)
masks = mask_generator.generate(image)

height, width, _ = image.shape
blank_image = np.zeros((height, width, 3), np.uint8)


plt.figure(figsize=(20, 20))
plt.imshow(blank_image)

show_anns(masks)
plt.axis('off')
plt.savefig('images/sam.png',bbox_inches='tight', pad_inches=0)

img_resize = cv2.imread('images/sam.png')
img_resize = cv2.resize(img_resize, (width, height))
cv2.imwrite('images/sam.png',img_resize)


"""
# write contour_coords.txt
"""
import numpy as np
import cv2

sorted_anns = sorted(masks, key=(lambda x: x['area']), reverse=True)

with open('contour_coords.txt', 'w') as file:
    for idx, ann in enumerate(sorted_anns):
        img = np.zeros((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 3))
        m = ann['segmentation']  # shape = height and weidth of input image (0-original)
        color_mask = np.concatenate([np.random.random(3)])
        img[m] = color_mask
        contours, _ = cv2.findContours((img[:, :, 2] > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        contour_coords = []

        if contours:
            main_contour = max(contours, key=cv2.contourArea)
        else:
            main_contour = None

        if main_contour is not None:
            for point in main_contour:
                x, y = point[0]
                contour_coords.append((x, y))

        file.write(f"Ann {idx + 1}:\n")
        for x, y in contour_coords:
            file.write(f"{x}, {y}\n")
